# Pairwise Experiments

### Setup

In [ ]:
# You can set them inline
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [1]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv()

True

### Task

Let's set up a new task! Here, we have a salesperson named Bob. Bob has a lot of deals, so he wants to summarize what happened in this deals based off of some meeting transcripts.

Bob is iterating on a few different prompts, that will give him nice, concise transcripts for his deals.

Bob has curated a dataset of his deal transcripts, let's go ahead and load that in. You can take a look at the dataset as well if you're curious! Note that this is not a golden dataset, there is no reference output here.

In [2]:
from langsmith import Client

client = Client()
dataset = client.clone_public_dataset(
  "https://smith.langchain.com/public/9078d2f1-7bef-4ba7-b795-210a17682ef9/d"
)

### Experiments

Now, let's run some experiments on this dataset using two different prompts. Let's add an evaluator that tries to score how good our summaries are!

In [3]:
from pydantic import BaseModel, Field
# from openai import OpenAI
from anthropic import Anthropic
import json
import re

anthropic_client = Anthropic()

SUMMARIZATION_SYSTEM_PROMPT = """You are a judge, aiming to score how well a summary summarizes the content of a transcript"""

SUMMARIZATION_HUMAN_PROMPT = """
[The Meeting Transcript] {transcript}
[The Start of Summarization] {summary} [The End of Summarization]"""

class SummarizationScore(BaseModel):
    score: int = Field(description="""A score from 1-5 ranking how good the summarization is for the provided transcript, with 1 being a bad summary, and 5 being a great summary""")
    
def summary_score_evaluator(inputs: dict, outputs: dict) -> list:
    completion = anthropic_client.messages.create(
        model="claude-3-5-haiku-latest",
        system=SUMMARIZATION_SYSTEM_PROMPT + " Return a JSON object with a single field 'score' containing an integer from 1-5.",
        messages=[
            {
                "role": "user",
                "content": SUMMARIZATION_HUMAN_PROMPT.format(
                    transcript=inputs["transcript"],
                    summary=outputs.get("output", "N/A"),
                )}
        ],
        max_tokens=1024,
        temperature=0
    )

    response_text = completion.content[0].text

    try:
        json_match = re.search(r'{.*?}', response_text, re.DOTALL)
        if json_match:
            parsed_response = json.loads(json_match.group(0))
            if 'score' in parsed_response:
                return {"key": "summary_score", "score": parsed_response["score"]}
    except:
        pass

    match = re.search(r'([1-5])', response_text)
    if match:
        summary_score = int(match.group(1))
        return {"key": "summary_score", "score": summary_score}

    return {"key": "summary_score", "score": 3}


First, we'll run our experiment with a good version of our prompt!

In [4]:
# Prompt One: Good Prompt!
def good_summarizer(inputs: dict):
    response = anthropic_client.messages.create(
        model="claude-3-5-haiku-latest",
        messages=[
            {
                "role": "user",
                "content": f"Concisely summarize this meeting in 3 sentences. Make sure to include all of the important events. Meeting: {inputs['transcript']}"
            }
        ],
        temperature=0,
        max_tokens=1024
    )
    return response.content[0].text

client.evaluate(
    good_summarizer,
    data=dataset,
    evaluators=[summary_score_evaluator],
    experiment_prefix="Good Summarizer"
)

/Users/manan/PycharmProjects/langsmith-course/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'Good Summarizer-ae14e9b6' at:
https://smith.langchain.com/o/e37fb4e5-4567-4839-b84a-654be8101cf2/datasets/61704357-5de3-405c-a92d-c4643430c905/compare?selectedSessions=5f5c1b04-2d22-4e94-bc00-1f33f1f9eaee




5it [00:34,  6.97s/it]


,inputs.transcript,outputs.output,error,feedback.summary_score,execution_time,example_id,id
0,"Bob and Ms. Thompson (NO DEAL): Bob: Hi, Ms. T...","Bob, a Ford Motors sales representative, met w...",None,5,2.905655,532c8c81-6d3e-4775-ae11-3890d5d4cfa5,fb28b6f3-2f62-47d7-8ca5-3fc37effff6e
1,Bob and Ms. Nguyen (NO DEAL): Bob: Good aftern...,Bob met with Ms. Nguyen to discuss potential v...,None,5,2.828761,65c35a35-6361-40e1-b5da-c6c690ae52c2,6c860582-c77c-41d1-93f3-67e988f642d3
2,Bob and Mr. Carter (CLOSED DEAL): Bob: Welcome...,Bob met with Mr. Carter to discuss trading in ...,None,5,2.856053,c4874cc6-737f-4550-a3de-50addaba733b,98aeed7e-6682-46a8-9f3c-9190b7b33cbc
3,"Bob and Mr. Patel (CLOSED DEAL): Bob: Hello, M...",Bob successfully helped Mr. Patel find a Ford ...,None,5,2.454100,cafc24bd-d1e5-41d5-b334-f2e0fd9e6872,ef7aa568-92f6-4526-b2d0-8421bb5f30c4
4,Bob and Mr. Johnson (CLOSED DEAL): Bob: Good m...,Bob introduced Mr. Johnson to the Ford Explore...,None,5,2.137282,e8c7eddc-bf4b-40af-84b8-916c7bbd965c,cb5b7448-091a-4228-b76f-3c0cdc7fdbc1


Now, we'll run an experiment with a worse version of our prompt, to highlight the difference.

In [5]:
# Prompt Two: Worse Prompt!
def bad_summarizer(inputs: dict):
    response = anthropic_client.messages.create(
        model="claude-3-5-haiku-latest",
        messages=[
            {
                "role": "user",
                "content": f"Summarize this in one sentence. {inputs['transcript']}"
            }
        ],
        temperature=0,
        max_tokens=1024
    )
    return response.content[0].text

client.evaluate(
    bad_summarizer,
    data=dataset,
    evaluators=[summary_score_evaluator],
    experiment_prefix="Bad Summarizer"
)

View the evaluation results for experiment: 'Bad Summarizer-293ca6c1' at:
https://smith.langchain.com/o/e37fb4e5-4567-4839-b84a-654be8101cf2/datasets/61704357-5de3-405c-a92d-c4643430c905/compare?selectedSessions=d0ab8e3f-1da0-4c91-b4e8-c1800cf7882a




5it [00:27,  5.54s/it]


,inputs.transcript,outputs.output,error,feedback.summary_score,execution_time,example_id,id
0,"Bob and Ms. Thompson (NO DEAL): Bob: Hi, Ms. T...","Bob, a Ford Motors salesperson, engages Ms. Th...",None,5,2.244849,532c8c81-6d3e-4775-ae11-3890d5d4cfa5,c14b9de0-ec3b-4154-96a0-1979760b2fe7
1,Bob and Ms. Nguyen (NO DEAL): Bob: Good aftern...,"Bob tries to sell Ms. Nguyen a Ford vehicle, b...",None,4,1.598144,65c35a35-6361-40e1-b5da-c6c690ae52c2,90847e3b-33a8-4ed4-85e8-30091647a041
2,Bob and Mr. Carter (CLOSED DEAL): Bob: Welcome...,Bob helps Mr. Carter trade in his 2015 Tacoma ...,None,5,1.599474,c4874cc6-737f-4550-a3de-50addaba733b,c77f7cd6-a298-4e80-b882-e9b56b17790f
3,"Bob and Mr. Patel (CLOSED DEAL): Bob: Hello, M...",Bob successfully helps Mr. Patel find and purc...,None,5,1.987214,cafc24bd-d1e5-41d5-b334-f2e0fd9e6872,c146ebe5-f444-4a4a-b539-c7b8c3f4f178
4,Bob and Mr. Johnson (CLOSED DEAL): Bob: Good m...,Bob successfully helps Mr. Johnson find and pu...,None,5,1.732207,e8c7eddc-bf4b-40af-84b8-916c7bbd965c,801c9d88-5866-4067-ac4f-d263c26e30ce


### Pairwise Experiment

Let's define a function that will compare our two experiments. These are the fields that pairwise evaluator functions get access to:
- `inputs: dict`: A dictionary of the inputs corresponding to a single example in a dataset.
- `outputs: list[dict]`: A list of the dict outputs produced by each experiment on the given inputs.
- `reference_outputs: dict`: A dictionary of the reference outputs associated with the example, if available.
- `runs: list[Run]`: A list of the full Run objects generated by the experiments on the given example. Use this if you need access to intermediate steps or metadata about each run.
- `example: Example`: The full dataset Example, including the example inputs, outputs (if available), and metdata (if available).

First, let's give our LLM-as-Judge some instructions. In our case, we're just going to directly use LLM-as-judge to grade which of the summarizers is the most helpful.

It might be hard to grade our summarizers without a ground truth reference, but here, comparing different prompts head to head will give us a sense of which is better!

In [6]:
JUDGE_SYSTEM_PROMPT = """
Please act as an impartial judge and evaluate the quality of the summarizations provided by two AI summarizers to the meeting transcript below.
Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of their summarizations. 
Begin your evaluation by comparing the two summarizations and provide a short explanation. 
Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision. 
Do not favor certain names of the assistants. 
Be as objective as possible. """

JUDGE_HUMAN_PROMPT = """
[The Meeting Transcript] {transcript}

[The Start of Assistant A's Summarization] {answer_a} [The End of Assistant A's Summarization]

[The Start of Assistant B's Summarization] {answer_b} [The End of Assistant B's Summarization]"""

Our function will take in an `inputs` dictionary, and a list of `outputs` dictionaries for the different experiments that we want to compare.

In [7]:
from pydantic import BaseModel, Field

class Preference(BaseModel):
    preference: int = Field(description="""1 if Assistant A answer is better based upon the factors above.
2 if Assistant B answer is better based upon the factors above.
Output 0 if it is a tie.""")
    
def ranked_preference(inputs: dict, outputs: list[dict]) -> list:
    completion = anthropic_client.messages.create(
        model="claude-3-5-haiku-latest",
        system=JUDGE_SYSTEM_PROMPT + " Return a JSON object with a single field 'preference' containing an integer: 1 if Assistant A is better, 2 if Assistant B is better, or 0 if it's a tie.",
        messages=[
            {
                "role": "user",
                "content": JUDGE_HUMAN_PROMPT.format(
                    transcript=inputs["transcript"],
                    answer_a=outputs[0].get("output", "N/A"),
                    answer_b=outputs[1].get("output", "N/A")
                )
            }
        ],
        max_tokens=1024,
        temperature=0
    )

    response_text = completion.content[0].text

    try:
        json_match = re.search(r'{.*?}', response_text, re.DOTALL)
        if json_match:
            parsed_response = json.loads(json_match.group(0))
            if 'preference' in parsed_response:
                preference_score = parsed_response["preference"]
            else:
                match = re.search(r'([0-2])', response_text)
                preference_score = int(match.group(1)) if match else 0
        else:
            match = re.search(r'([0-2])', response_text)
            preference_score = int(match.group(1)) if match else 0
    except:
        match = re.search(r'([0-2])', response_text)
        preference_score = int(match.group(1)) if match else 0

    if preference_score == 1:
        scores = [1, 0]
    elif preference_score == 2:
        scores = [0, 1]
    else:
        scores = [0, 0]
    return scores


Now let's run our pairwise experiment with `evaluate()`

In [8]:
from langsmith import evaluate

evaluate(
    ("Good Summarizer-ae14e9b6", "Bad Summarizer-293ca6c1"),
    evaluators=[ranked_preference]
)

View the pairwise evaluation results at:
https://smith.langchain.com/o/e37fb4e5-4567-4839-b84a-654be8101cf2/datasets/61704357-5de3-405c-a92d-c4643430c905/compare?selectedSessions=5f5c1b04-2d22-4e94-bc00-1f33f1f9eaee%2Cd0ab8e3f-1da0-4c91-b4e8-c1800cf7882a&comparativeExperiment=53bc2af2-00f6-4bdf-84c0-b998ff7bfbd6




100%|██████████| 5/5 [00:28<00:00,  5.66s/it]
